# Branching with Imitation Learning and a GNN

In this tutorial we will reproduce a simplified version of the paper of Gasse et al. (2019) on learning to branch with Ecole with `pytorch` and `pytorch geometric`. We collect strong branching examples on randomly generated maximum set covering instances, then train a graph neural network with bipartite state encodings to imitate the expert by classification. Finally, we will evaluate the quality of the policy.

The biggest difference with Gasse et al. (2019) is that only n=1,000 training examples of expert decisions are collected for training, to keep the time needed to run the tutorial reasonable. As a consequence, the resulting policy is undertrained and is not competitive with SCIP's default branching rule.

Users that are interested in reproducing competitive performance should use a larger sample size, such as the n=100,000 samples used for training in the paper. In this case, we strongly recommend to parallelize data collection, as in the original Gasse et al. (2019) code.

### Requirements
The requirements can be found in `conda-requirements.yaml`, lock files with pinned versions are also available
for various configurations.

In [1]:
import gzip
import pickle
from pathlib import Path

import ecole
import numpy as np
import torch
import torch.nn.functional as F
import torch_geometric

In [2]:
DATA_MAX_SAMPLES = 100
LEARNING_RATE = 0.001
NB_EPOCHS = 50
NB_EVAL_INSTANCES = 20

## 1. Data collection

Our first step will be to run explore-then-strong-branch on randomly generated maximum set covering instances, and save the branching decisions to build a dataset. We will also record the state of the branch-and-bound process as a bipartite graph, which is already implemented in Ecole with the same features as Gasse et al. (2019).

We will use the Ecole-provided set cover instance generator.

In [4]:
#instances = ecole.instance.SetCoverGenerator(n_rows=500, n_cols=1000, density=0.05)
instances = ecole.instance.FileGenerator()


The explore-then-strong-branch scheme described in the paper is not implemented by default in Ecole. In this scheme, to diversify the states in which we collect examples of strong branching behavior, we mostly follow a weak but cheap expert (pseudocost branching) and only occasionally call the strong expert (strong branching). This also ensures that samples are closer to being independent and identically distributed.

This can be realized in Ecole by creating a custom observation function, which will randomly compute and return the pseudocost scores (cheap) or the strong branching scores (expensive). It also showcases extensibility in Ecole by showing how easily a custom observation function can be created and used, directly in Python.

In [4]:
class ExploreThenStrongBranch:
    """
    This custom observation function class will randomly return either strong branching scores (expensive expert)
    or pseudocost scores (weak expert for exploration) when called at every node.
    """

    def __init__(self, expert_probability):
        self.expert_probability = expert_probability
        self.pseudocosts_function = ecole.observation.Pseudocosts()
        self.strong_branching_function = ecole.observation.StrongBranchingScores()

    def before_reset(self, model):
        """
        This function will be called at initialization of the environment (before dynamics are reset).
        """
        self.pseudocosts_function.before_reset(model)
        self.strong_branching_function.before_reset(model)

    def extract(self, model, done):
        """
        Should we return strong branching or pseudocost scores at time node?
        """
        probabilities = [1 - self.expert_probability, self.expert_probability]
        expert_chosen = bool(np.random.choice(np.arange(2), p=probabilities)) # randoom choose a probability
        if expert_chosen:
            return (self.strong_branching_function.extract(model, done), True) # extract chosen in strong branch 
        else:
            return (self.pseudocosts_function.extract(model, done), False) # extract in pseudocosts

We can now create the environment with the correct parameters (no restarts, 1h time limit, 5% expert sampling probability).

Besides the (pseudocost or strong branching) scores, our environment will return the node bipartite graph representation of 
branch-and-bound states used in Gasse et al. (2019), using the `ecole.observation.NodeBipartite` observation function.
On one side of that bipartite graph, nodes represent the variables of the problem, with a vector encoding features of 
that variable. On the other side of the bipartite graph, nodes represent the constraints of the problem, similarly with 
a vector encoding features of that constraint. An edge links a variable and a constraint node if the variable participates 
in that constraint, that is, its coefficient is nonzero in that constraint. The constraint coefficient is attached as an
attribute of the edge.

In [5]:
# We can pass custom SCIP parameters easily
scip_parameters = {
    "separating/maxrounds": 0,
    "presolving/maxrestarts": 0,
    "limits/time": 3600, # 1h time
}

# Note how we can tuple observation functions to return complex state information
env = ecole.environment.Branching(
    observation_function=(
        ExploreThenStrongBranch(expert_probability=0.05), # 5% times expert decision
        ecole.observation.NodeBipartite(),
    ),
    scip_params=scip_parameters,
)

# This will seed the environment for reproducibility
env.seed(0)

Now we loop over the instances, following the strong branching expert 5% of the time and saving its decision, until enough samples are collected.

In [9]:
episode_counter, sample_counter = 0, 0
Path("samples_100/").mkdir(exist_ok=True)

# We will solve problems (run episodes) until we have saved enough samples
while sample_counter < DATA_MAX_SAMPLES:
    episode_counter += 1

    observation, action_set, _, done, _ = env.reset(next(instances))
    while not done:
        (scores, scores_are_expert), node_observation = observation
        action = action_set[scores[action_set].argmax()]

        # Only save samples if they are coming from the expert (strong branching)
        if scores_are_expert and (sample_counter < DATA_MAX_SAMPLES):
            sample_counter += 1
            data = [node_observation, action, action_set, scores]
            filename = f"samples_100/sample_{sample_counter}.pkl"

            with gzip.open(filename, "wb") as f:
                pickle.dump(data, f)

        observation, action_set, _, done, _ = env.step(action)

    print(f"Episode {episode_counter}, {sample_counter} samples collected so far")

Episode 1, 1 samples collected so far
Episode 2, 1 samples collected so far
Episode 3, 8 samples collected so far
Episode 4, 10 samples collected so far
Episode 5, 11 samples collected so far
Episode 6, 17 samples collected so far
Episode 7, 23 samples collected so far
Episode 8, 23 samples collected so far
Episode 9, 25 samples collected so far
Episode 10, 25 samples collected so far
Episode 11, 25 samples collected so far
Episode 12, 25 samples collected so far
Episode 13, 25 samples collected so far
Episode 14, 25 samples collected so far
Episode 15, 26 samples collected so far
Episode 16, 28 samples collected so far
Episode 17, 37 samples collected so far
Episode 18, 38 samples collected so far
Episode 19, 39 samples collected so far
Episode 20, 39 samples collected so far
Episode 21, 42 samples collected so far
Episode 22, 47 samples collected so far
Episode 23, 47 samples collected so far
Episode 24, 50 samples collected so far
Episode 25, 79 samples collected so far
Episode 26, 

In [6]:
# Look a how samples are stored

with gzip.open("samples_100/sample_10.pkl", "rb") as f:
        sample = pickle.load(f)
        sample_observation, sample_action, sample_action_set, sample_scores = sample


In [8]:
sample_observation.column_features

array([[0.0124701 , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05259128, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00379525, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03903682, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04120554, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03144633, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
sample_action


254

In [9]:
sample_action_set

array([ 38,  70,  81,  82, 121, 130, 151, 168, 172, 173, 179, 223, 254,
       260, 272, 280, 293, 353, 362, 364, 368, 377, 389, 405, 413, 414,
       415, 425, 435, 440, 499, 506, 509, 525, 550, 560, 580, 589, 609,
       612, 615, 642, 649, 710, 715, 731, 734, 807, 820, 821, 822, 829,
       837, 844, 867, 904, 924, 926, 943, 965], dtype=uint64)

In [10]:
print(f'In Sample Scores we have {sample_scores.shape} array, but most are Nan')
sample_scores[~np.isnan(sample_scores)]


In Sample Scores we have (990,) array, but most are Nan


array([1.01428440e-06, 1.43870127e-02, 1.01428440e-06, 7.29027614e-04,
       1.86498313e-03, 5.67633048e-01, 1.02867141e+00, 8.27449776e-02,
       4.58136436e-01, 2.93029597e-02, 2.59671475e-01, 5.72243431e-01,
       3.72369741e-01, 1.01428440e-06, 8.37529669e-01, 1.02877284e+00,
       6.28355173e-01, 2.94933761e-01, 7.51221434e-07, 1.43870127e-02,
       1.01428440e-06, 4.40556928e-01, 1.02881625e-01, 7.61892878e-02,
       1.02877284e+00, 6.88511297e-01, 5.49583886e-02, 1.02867141e+00,
       1.01428440e-06, 2.04712743e-01, 2.47173596e-01, 2.94933761e-01,
       7.24386091e-01, 6.32176754e-01, 1.02867141e+00, 1.01428440e-06,
       1.01428440e-06, 1.01428440e-06, 2.33492137e-01, 1.87031165e-01,
       7.15165323e-01, 2.22132010e-01, 1.01428440e-06, 1.01428440e-06,
       6.22957651e-01, 3.04182555e-01, 5.21529211e-01, 1.01428440e-06,
       1.43870127e-02, 1.01428440e-06, 3.30193735e-03, 2.48452643e-01,
       6.24953636e-02, 4.97379583e-01, 1.01428440e-06, 4.75425375e-01,
      

# 2. Train a GNN

Our next step is to train a GNN classifier on these collected samples to predict similar choices to strong branching.

In [13]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

We will first define pytorch geometric data classes to handle the bipartite graph data.

In [14]:
class BipartiteNodeData(torch_geometric.data.Data):
    """
    This class encode a node bipartite graph observation as returned by the `ecole.observation.NodeBipartite`
    observation function in a format understood by the pytorch geometric data handlers.
    """

    def __init__(
        self,
        constraint_features,
        edge_indices,
        edge_features,
        variable_features,
        candidates,
        nb_candidates,
        candidate_choice,
        candidate_scores,
    ):
        super().__init__()
        self.constraint_features = constraint_features
        self.edge_index = edge_indices
        self.edge_attr = edge_features
        self.variable_features = variable_features
        self.candidates = candidates
        self.nb_candidates = nb_candidates
        self.candidate_choices = candidate_choice
        self.candidate_scores = candidate_scores

    def __inc__(self, key, value, store, *args, **kwargs):
        """
        We overload the pytorch geometric method that tells how to increment indices when concatenating graphs
        for those entries (edge index, candidates) for which this is not obvious.
        """
        if key == "edge_index":
            return torch.tensor(
                [[self.constraint_features.size(0)], [self.variable_features.size(0)]]
            )
        elif key == "candidates":
            return self.variable_features.size(0)
        else:
            return super().__inc__(key, value, *args, **kwargs)


class GraphDataset(torch_geometric.data.Dataset):
    """
    This class encodes a collection of graphs, as well as a method to load such graphs from the disk.
    It can be used in turn by the data loaders provided by pytorch geometric.
    """

    def __init__(self, sample_files):
        super().__init__(root=None, transform=None, pre_transform=None)
        self.sample_files = sample_files

    def len(self):
        return len(self.sample_files)

    def get(self, index):
        """
        This method loads a node bipartite graph observation as saved on the disk during data collection.
        """
        with gzip.open(self.sample_files[index], "rb") as f:
            sample = pickle.load(f)

        sample_observation, sample_action, sample_action_set, sample_scores = sample
        
        constraint_features = sample_observation.row_features
        edge_indices = sample_observation.edge_features.indices.astype(np.int32)
        edge_features = np.expand_dims(sample_observation.edge_features.values, axis=-1)
        variable_features = sample_observation.column_features #DEBUG, IT WAS variable_features

        # We note on which variables we were allowed to branch, the scores as well as the choice
        # taken by strong branching (relative to the candidates)
        candidates = np.array(sample_action_set, dtype=np.int32)
        candidate_scores = np.array([sample_scores[j] for j in candidates])
        candidate_choice = np.where(candidates == sample_action)[0][0]

        graph = BipartiteNodeData(
            torch.FloatTensor(constraint_features),
            torch.LongTensor(edge_indices),
            torch.FloatTensor(edge_features),
            torch.FloatTensor(variable_features),
            torch.LongTensor(candidates),
            len(candidates),
            torch.LongTensor([candidate_choice]),
            torch.FloatTensor(candidate_scores)
        )

        # We must tell pytorch geometric how many nodes there are, for indexing purposes
        graph.num_nodes = constraint_features.shape[0] + variable_features.shape[0]

        return graph

In [15]:
sample_observation.column_features

array([[0.0124701 , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05259128, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.00379525, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.03903682, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.04120554, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03144633, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

We can then prepare the data loaders.

In [16]:
sample_files = [str(path) for path in Path("samples_100/").glob("sample_*.pkl")]
train_files = sample_files[: int(0.8 * len(sample_files))]
valid_files = sample_files[int(0.8 * len(sample_files)) :]

train_data = GraphDataset(train_files)
train_loader = torch_geometric.loader.DataLoader(train_data, batch_size=32, shuffle=True)
valid_data = GraphDataset(valid_files)
valid_loader = torch_geometric.loader.DataLoader(valid_data, batch_size=128, shuffle=False)

Next, we will define our graph neural network architecture.

In [17]:
class GNNPolicy(torch.nn.Module):
    def __init__(self):
        super().__init__()
        emb_size = 64
        cons_nfeats = 5
        edge_nfeats = 1
        var_nfeats = 19

        # CONSTRAINT EMBEDDING
        self.cons_embedding = torch.nn.Sequential(
            torch.nn.LayerNorm(cons_nfeats),
            torch.nn.Linear(cons_nfeats, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
            torch.nn.ReLU(),
        )

        # EDGE EMBEDDING
        self.edge_embedding = torch.nn.Sequential(
            torch.nn.LayerNorm(edge_nfeats),
        )

        # VARIABLE EMBEDDING
        self.var_embedding = torch.nn.Sequential(
            torch.nn.LayerNorm(var_nfeats),
            torch.nn.Linear(var_nfeats, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
            torch.nn.ReLU(),
        )

        self.conv_v_to_c = BipartiteGraphConvolution()
        self.conv_c_to_v = BipartiteGraphConvolution()

        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, 1, bias=False),
        )

    def forward(
        self, constraint_features, edge_indices, edge_features, variable_features
    ):
        reversed_edge_indices = torch.stack([edge_indices[1], edge_indices[0]], dim=0)

        # First step: linear embedding layers to a common dimension (64)
        constraint_features = self.cons_embedding(constraint_features)
        edge_features = self.edge_embedding(edge_features)
        variable_features = self.var_embedding(variable_features)

        # Two half convolutions
        constraint_features = self.conv_v_to_c(
            variable_features, reversed_edge_indices, edge_features, constraint_features
        )
        variable_features = self.conv_c_to_v(
            constraint_features, edge_indices, edge_features, variable_features
        )

        # A final MLP on the variable features
        output = self.output_module(variable_features).squeeze(-1)
        return output


class BipartiteGraphConvolution(torch_geometric.nn.MessagePassing):
    """
    The bipartite graph convolution is already provided by pytorch geometric and we merely need
    to provide the exact form of the messages being passed.
    """

    def __init__(self):
        super().__init__("add")
        emb_size = 64

        self.feature_module_left = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size)
        )
        self.feature_module_edge = torch.nn.Sequential(
            torch.nn.Linear(1, emb_size, bias=False)
        )
        self.feature_module_right = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size, bias=False)
        )
        self.feature_module_final = torch.nn.Sequential(
            torch.nn.LayerNorm(emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )

        self.post_conv_module = torch.nn.Sequential(torch.nn.LayerNorm(emb_size))

        # output_layers
        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(2 * emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )

    def forward(self, left_features, edge_indices, edge_features, right_features):
        """
        This method sends the messages, computed in the message method.
        """
        output = self.propagate(
            edge_indices,
            size=(left_features.shape[0], right_features.shape[0]),
            node_features=(left_features, right_features),
            edge_features=edge_features,
        )
        return self.output_module(
            torch.cat([self.post_conv_module(output), right_features], dim=-1)
        )

    def message(self, node_features_i, node_features_j, edge_features):
        output = self.feature_module_final(
            self.feature_module_left(node_features_i)
            + self.feature_module_edge(edge_features)
            + self.feature_module_right(node_features_j)
        )
        return output


policy = GNNPolicy().to(DEVICE)

With this model we can predict a probability distribution over actions as follows.

In [18]:
observation = train_data[0].to(DEVICE)

logits = policy(
    observation.constraint_features,
    observation.edge_index,
    observation.edge_attr,
    observation.variable_features,
)
action_distribution = F.softmax(logits[observation.candidates], dim=-1)

print(action_distribution) # probs sums to 1

tensor([0.0121, 0.0122, 0.0122, 0.0122, 0.0122, 0.0121, 0.0122, 0.0122, 0.0121,
        0.0121, 0.0121, 0.0123, 0.0122, 0.0121, 0.0121, 0.0121, 0.0122, 0.0122,
        0.0121, 0.0122, 0.0122, 0.0121, 0.0121, 0.0122, 0.0122, 0.0121, 0.0122,
        0.0123, 0.0122, 0.0122, 0.0123, 0.0122, 0.0122, 0.0121, 0.0123, 0.0122,
        0.0122, 0.0122, 0.0121, 0.0121, 0.0121, 0.0122, 0.0122, 0.0121, 0.0121,
        0.0121, 0.0122, 0.0123, 0.0122, 0.0121, 0.0122, 0.0123, 0.0122, 0.0123,
        0.0123, 0.0123, 0.0123, 0.0121, 0.0121, 0.0121, 0.0123, 0.0123, 0.0122,
        0.0122, 0.0122, 0.0123, 0.0121, 0.0122, 0.0123, 0.0123, 0.0123, 0.0122,
        0.0122, 0.0121, 0.0121, 0.0123, 0.0122, 0.0123, 0.0122, 0.0123, 0.0122,
        0.0123], grad_fn=<SoftmaxBackward0>)


As can be seen, with randomly initialized weights, the initial distributions tend to be close to uniform.
Next, we will define two helper functions: one to train or evaluate the model on a whole epoch and compute metrics for monitoring, and one for padding tensors when doing predictions on a batch of graphs of potentially different number of variables.

In [22]:
def process(policy, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    mean_loss = 0
    mean_acc = 0

    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for batch in data_loader:
            batch = batch.to(DEVICE)
            # Compute the logits (i.e. pre-softmax activations) according to the policy on the concatenated graphs
            logits = policy(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
            )
            # Index the results by the candidates, and split and pad them
            logits = pad_tensor(logits[batch.candidates], batch.nb_candidates)
            # Compute the usual cross-entropy classification loss
            loss = F.cross_entropy(logits, batch.candidate_choices)

            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            true_scores = pad_tensor(batch.candidate_scores, batch.nb_candidates)
            true_bestscore = true_scores.max(dim=-1, keepdims=True).values

            predicted_bestindex = logits.max(dim=-1, keepdims=True).indices
            accuracy = (
                (true_scores.gather(-1, predicted_bestindex) == true_bestscore)
                .float()
                .mean()
                .item()
            )

            mean_loss += loss.item() * batch.num_graphs
            mean_acc += accuracy * batch.num_graphs
            n_samples_processed += batch.num_graphs

    mean_loss /= n_samples_processed
    mean_acc /= n_samples_processed
    return mean_loss, mean_acc


def pad_tensor(input_, pad_sizes, pad_value=-1e8):
    """
    This utility function splits a tensor and pads each split to make them all the same size, then stacks them.
    """
    max_pad_size = pad_sizes.max()
    output = input_.split(pad_sizes.cpu().numpy().tolist())
    output = torch.stack(
        [
            F.pad(slice_, (0, max_pad_size - slice_.size(0)), "constant", pad_value)
            for slice_ in output
        ],
        dim=0,
    )
    return output

After this, we can actually create the model and train it.

In [23]:
optimizer = torch.optim.Adam(policy.parameters(), lr=LEARNING_RATE)
for epoch in range(NB_EPOCHS):
    print(f"Epoch {epoch+1}")

    train_loss, train_acc = process(policy, train_loader, optimizer)
    print(f"Train loss: {train_loss:0.3f}, accuracy {train_acc:0.3f}")

    valid_loss, valid_acc = process(policy, valid_loader, None)
    print(f"Valid loss: {valid_loss:0.3f}, accuracy {valid_acc:0.3f}")

torch.save(policy.state_dict(), "trained_params.pkl")

Epoch 1
Train loss: 4.283, accuracy 0.150
Valid loss: 4.242, accuracy 0.100
Epoch 2
Train loss: 4.277, accuracy 0.100
Valid loss: 4.233, accuracy 0.250
Epoch 3
Train loss: 4.259, accuracy 0.138
Valid loss: 4.201, accuracy 0.350
Epoch 4
Train loss: 4.198, accuracy 0.188
Valid loss: 4.145, accuracy 0.600
Epoch 5
Train loss: 4.100, accuracy 0.325
Valid loss: 4.043, accuracy 0.600
Epoch 6
Train loss: 3.974, accuracy 0.338
Valid loss: 3.970, accuracy 0.500
Epoch 7
Train loss: 3.823, accuracy 0.375
Valid loss: 3.805, accuracy 0.550
Epoch 8
Train loss: 3.668, accuracy 0.338
Valid loss: 3.792, accuracy 0.550
Epoch 9
Train loss: 3.642, accuracy 0.412
Valid loss: 3.681, accuracy 0.600
Epoch 10
Train loss: 3.513, accuracy 0.388
Valid loss: 3.915, accuracy 0.600
Epoch 11
Train loss: 3.417, accuracy 0.388
Valid loss: 3.546, accuracy 0.550
Epoch 12
Train loss: 3.316, accuracy 0.438
Valid loss: 3.848, accuracy 0.600
Epoch 13
Train loss: 3.385, accuracy 0.450
Valid loss: 3.508, accuracy 0.600
Epoch 14

# 3 Evaluation

Finally, we can evaluate the performance of the model. We first define appropriate environments. For benchmarking purposes, we include a trivial environment that merely runs SCIP.

In [24]:
scip_parameters = {
    "separating/maxrounds": 0,
    "presolving/maxrestarts": 0,
    "limits/time": 3600,
}
env = ecole.environment.Branching(
    observation_function=ecole.observation.NodeBipartite(),
    information_function={
        "nb_nodes": ecole.reward.NNodes(),
        "time": ecole.reward.SolvingTime(),
    },
    scip_params=scip_parameters,
)
default_env = ecole.environment.Configuring(
    observation_function=None,
    information_function={
        "nb_nodes": ecole.reward.NNodes(),
        "time": ecole.reward.SolvingTime(),
    },
    scip_params=scip_parameters,
)

Then we can simply follow the environments, taking steps appropriately according to the GNN policy.

In [26]:
instances = ecole.instance.SetCoverGenerator(n_rows=500, n_cols=1000, density=0.05)
for instance_count, instance in zip(range(NB_EVAL_INSTANCES), instances):
    
    # Run the GNN brancher
    nb_nodes, time = 0, 0
    observation, action_set, _, done, info = env.reset(instance)
    nb_nodes += info["nb_nodes"]
    time += info["time"]
    while not done:
        with torch.no_grad():
            observation = (
                torch.from_numpy(observation.row_features.astype(np.float32)).to(DEVICE),
                torch.from_numpy(observation.edge_features.indices.astype(np.int64)).to(DEVICE),
                torch.from_numpy(observation.edge_features.values.astype(np.float32)).view(-1, 1).to(DEVICE),
                torch.from_numpy(observation.column_features.astype(np.float32)).to(DEVICE),
            )
            logits = policy(*observation)
            action = action_set[logits[action_set.astype(np.int64)].argmax()]
            observation, action_set, _, done, info = env.step(action)
        nb_nodes += info["nb_nodes"]
        time += info["time"]

    # Run SCIP's default brancher
    default_env.reset(instance)
    _, _, _, _, default_info = default_env.step({})

    print(f"Instance {instance_count: >3} | SCIP nb nodes    {int(default_info['nb_nodes']): >4d}  | SCIP time   {default_info['time']: >6.2f} ")
    print(f"             | GNN  nb nodes    {int(nb_nodes): >4d}  | GNN  time   {time: >6.2f} ")
    print(f"             | Gain         {100*(1-nb_nodes/default_info['nb_nodes']): >8.2f}% | Gain      {100*(1-time/default_info['time']): >8.2f}%")

Instance   0 | SCIP nb nodes       1  | SCIP time     1.25 
             | GNN  nb nodes       5  | GNN  time     1.27 
             | Gain          -400.00% | Gain         -1.45%
Instance   1 | SCIP nb nodes       3  | SCIP time     1.75 
             | GNN  nb nodes      21  | GNN  time     2.07 
             | Gain          -600.00% | Gain        -17.77%
Instance   2 | SCIP nb nodes       1  | SCIP time     1.78 
             | GNN  nb nodes      17  | GNN  time     1.93 
             | Gain         -1600.00% | Gain         -8.23%
Instance   3 | SCIP nb nodes    1257  | SCIP time    13.58 
             | GNN  nb nodes    11077  | GNN  time   581.57 
             | Gain          -781.23% | Gain      -4181.01%
Instance   4 | SCIP nb nodes       1  | SCIP time     2.72 
             | GNN  nb nodes      23  | GNN  time     3.15 
             | Gain         -2200.00% | Gain        -15.87%
Instance   5 | SCIP nb nodes      11  | SCIP time     4.29 
             | GNN  nb nodes     141  |

We can also evaluate on instances larger and harder than those trained on, say with 600 rather than 500 constraints.
In addition, we showcase that the cumulative number of nodes and time required to solve an instance can also be computed directly using the `.cumsum()` method.

In [16]:
instances = ecole.instance.SetCoverGenerator(n_rows=600, n_cols=1000, density=0.05)
scip_parameters = {
    "separating/maxrounds": 0,
    "presolving/maxrestarts": 0,
    "limits/time": 3600,
}
env = ecole.environment.Branching(
    observation_function=ecole.observation.NodeBipartite(),
    information_function={
        "nb_nodes": ecole.reward.NNodes().cumsum(),
        "time": ecole.reward.SolvingTime().cumsum(),
    },
    scip_params=scip_parameters,
)
default_env = ecole.environment.Configuring(
    observation_function=None,
    information_function={
        "nb_nodes": ecole.reward.NNodes().cumsum(),
        "time": ecole.reward.SolvingTime().cumsum(),
    },
    scip_params=scip_parameters,
)

for instance_count, instance in zip(range(NB_EVAL_INSTANCES), instances):
    # Run the GNN brancher
    observation, action_set, _, done, info = env.reset(instance)
    while not done:
        with torch.no_grad():
            observation = (
                torch.from_numpy(observation.row_features.astype(np.float32)).to(DEVICE),
                torch.from_numpy(observation.edge_features.indices.astype(np.int64)).to(DEVICE),
                torch.from_numpy(observation.edge_features.values.astype(np.float32)).view(-1, 1).to(DEVICE),
                torch.from_numpy(observation.variable_features.astype(np.float32)).to(DEVICE),
            )
            logits = policy(*observation)
            action = action_set[logits[action_set.astype(np.int64)].argmax()]
            observation, action_set, _, done, info = env.step(action)
    nb_nodes = info["nb_nodes"]
    time = info["time"]

    # Run SCIP's default brancher
    default_env.reset(instance)
    _, _, _, _, default_info = default_env.step({})

    print(
        f"Instance {instance_count: >3} | SCIP nb nodes    {int(default_info['nb_nodes']): >4d}  | SCIP time   {default_info['time']: >6.2f} "
    )
    print(
        f"             | GNN  nb nodes    {int(nb_nodes): >4d}  | GNN  time   {time: >6.2f} "
    )
    print(
        f"             | Gain         {100*(1-nb_nodes/default_info['nb_nodes']): >8.2f}% | Gain      {100*(1-time/default_info['time']): >8.2f}%"
    )

Instance   0 | SCIP nb nodes      29  | SCIP time     3.21 
             | GNN  nb nodes     113  | GNN  time     2.12 
             | Gain          -289.66% | Gain         34.19%
Instance   1 | SCIP nb nodes       9  | SCIP time     3.32 
             | GNN  nb nodes      93  | GNN  time     2.57 
             | Gain          -933.33% | Gain         22.41%
Instance   2 | SCIP nb nodes      17  | SCIP time     4.60 
             | GNN  nb nodes     103  | GNN  time     3.13 
             | Gain          -505.88% | Gain         31.89%
Instance   3 | SCIP nb nodes       3  | SCIP time     2.63 
             | GNN  nb nodes      19  | GNN  time     1.89 
             | Gain          -533.33% | Gain         28.39%
Instance   4 | SCIP nb nodes       7  | SCIP time     2.88 
             | GNN  nb nodes      52  | GNN  time     2.33 
             | Gain          -642.86% | Gain         19.36%
Instance   5 | SCIP nb nodes     201  | SCIP time     5.39 
             | GNN  nb nodes     569  | 

### References

Gasse, M., Chételat, D., Ferroni, N., Charlin, L. and Lodi, A. (2019). Exact combinatorial optimization with graph convolutional neural networks. In Advances in Neural Information Processing Systems (pp. 15580-15592).